In [67]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from decimal import Decimal
from datetime import datetime

%matplotlib inline

import findspark
findspark.init()

from pyspark.sql import *
from pyspark.sql.functions import  isnan, when, count, col,isnull

%matplotlib inline

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [58]:
spark

In [9]:
sns.set_context('talk')
%matplotlib inline

sns.set_style("darkgrid", {"axes.facecolor": ".9"})
sns.set_context('notebook')

In [10]:
data_folder = './data/'

In [88]:
data  = spark.read.csv(data_folder+'en.openfoodfacts.org.products.csv',header = True, sep="\t")

In [84]:
selected = data.select(['code','countries','categories','origins','nutrition-score-fr_100g','packaging','-palmitic-acid_100g'])

In [85]:
selected.count()

685395

In [96]:
ciao = data.select([(count(when(isnull(c), c))).alias(c) for c in data.columns])
ciaone = ciao.toPandas()

In [102]:
pd.options.display.max_columns = 100000000
ciaone[ciaone<300000].dropna(axis=1)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,brands,brands_tags,countries,countries_tags,countries_en,ingredients_text,serving_quantity,additives_n,additives,ingredients_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_n,states,states_tags,states_en,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,sodium_100g
0,34,34,3,4,10,0,0,26698,224276,224316,548,548,548,294125,18772,294159,294172,294159,294159,68,68,68,100119,105591,131379,105837,116107,101555,121352,121388
